In [1]:
from tic_tac_toe import UltimateBoard, ClassicBoard

import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%pylab inline

import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select

from itertools import product
from operator import itemgetter
import pickle

colors=['#547fc4']

Populating the interactive namespace from numpy and matplotlib


In [6]:
import time

class WebGame():
    
    def __init__(self, bot_lbl='O', vs_cpu=False, wait_time=1):
        self.vs_cpu = vs_cpu
        self.driver = self._set_up_game()
        self.board = UltimateBoard()
        self.moves = 0
        self.bot_lbl = 'X' if vs_cpu else bot_lbl
        self.wait_time = wait_time
        
        # generate board strings for all positions in the game
        # like 'tile0square0', 'tile0square1', ..., 'tile0square8','tile1square0',...,'tile8square8'  
        self.board_strings = [f'tile{board}square{pos}' for board,pos in product(range(9), range(9))]
        
    
    def _set_up_game(self):
        driver = webdriver.Chrome()
        driver.get('http://ultimatetictactoe.creativitygames.net/')
        
        # set game to be win by 3 in a row
        game_type = Select(driver.find_element_by_id('gametype'))
        try:
            game_type.select_by_visible_text("3 Tiles in a Row")
        except:
            pass
        
        # set up game against computer if necessary
        if self.vs_cpu:
            num_players = Select(driver.find_element_by_id('opponent'))
            try:
                num_players.select_by_visible_text("VS Computer")
            except:
                pass

        driver.find_element_by_id('newgame').click()

        return driver
    
    def move(self, board, pos, label):
        time.sleep(self.wait_time)
        move_id = f'tile{board}square{pos}'
        self.driver.find_element_by_id(move_id).click()
        
        self.moves += 1
        
    def watch_game(self):
        '''
        Function to capture the moves being played on a board in another window.
        Tracks state of board in game and compares it to the state of the boad
        object built on an UltimateBoard.
        '''
        
        runs = 0 
        tic = time.time()
        moves = 0
        if self.vs_cpu:
            brd, sq, prob = self.board.make_move('X')
            self.move(brd, sq, 'X')
            moves += 1
            
        while not self.board.game_over:
            for bs in self.board_strings:
                # bs like tile[x]square[y], so x is board y is pos
                brd_id = int(bs[4])
                pos = int(bs[-1])

                pos_i, pos_j = math.floor(pos / 3), pos % 3
                sq_str = self.driver.find_element_by_id(bs).get_attribute('class')
                
                # detect whether a change has been made on the board
                if len(sq_str.split()) > 2:
                    moves += 1
                    sq_stat = sq_str.split()[2]
                    
                    # Update an X play
                    if sq_stat == 'cross' and self.board.boards[brd_id].board[pos_i, pos_j] == 0:
                        self.board.play(brd_id, pos, 'X')
                        
                        print(f'-------Move {moves}-------')
                        print(self.board)
                        print('-------------------------')
                        print()
                        
                        if self.bot_lbl == 'O' and not self.board.game_over:
                            brd, sq, prob = self.board.make_move('O')
                            self.move(brd, sq, 'O')
                            print(brd, sq)
                                
                    # Update an O play
                    if sq_stat == 'naught' and self.board.boards[brd_id].board[pos_i, pos_j] == 0:
                        self.board.play(brd_id, pos, 'O')
                        
                        print(f'-------Move {moves}-------')
                        print(self.board)
                        print('-------------------------')
                        print()
                        
                        if self.bot_lbl == 'X' and not self.board.game_over:
                            brd, sq, prob = self.board.make_move('X')
                            self.move(brd, sq, 'X')
                            print(brd, sq)
                            
                runs += 1
                toc = time.time()
        
        print(f"Game Over - {board.winner} won!!")
    
    def __str__(self):
        return self.board.__str__()
        

In [7]:
game = WebGame(bot_lbl='O', vs_cpu=False, wait_time=2)

In [8]:
game.watch_game()

-------Move 10-------
- - -|- - -|- - -
- - -|- - -|- - -
- - -|- - -|- - -
-----------------
- - -|- - -|- - X
- - -|- - -|- - -
- - -|- - -|- - -
-----------------
- - -|- - -|- - -
- - -|- - -|- - -
- - -|- - -|- - -
-------------------------

2 7
-------Move 20-------
- - -|- - -|- - -
- - -|- - -|- - -
- - -|- - -|- O -
-----------------
- - -|- - -|- - X
- - -|- - -|- - -
- - -|- - -|- - -
-----------------
- - -|- - -|- - -
- - -|- - -|- - -
- - -|- - -|- - -
-------------------------

-------Move 143-------
- - -|- - -|- - -
- - -|- - -|- - -
- - -|- - -|- O -
-----------------
- - -|- - -|- - X
- - -|- - -|- - -
- - -|- - -|- - -
-----------------
- - -|- - -|- - -
- - -|X - -|- - -
- - -|- - -|- - -
-------------------------

3 8
-------Move 154-------
- - -|- - -|- - -
- - -|- - -|- - -
- - -|- - -|- O -
-----------------
- - -|- - -|- - X
- - -|- - -|- - -
- - O|- - -|- - -
-----------------
- - -|- - -|- - -
- - -|X - -|- - -
- - -|- - -|- - -
-------------------------

--

7 8
-------Move 3803-------
- - -|X - -|O - -
X O O|- X -|- O -
O - O|- - -|O O -
-----------------
X O X|- - X|- O X
O X -|O - X|O X -
- - O|O - X|- - -
-----------------
- - -|- - -|X - X
X X X|X - -|- O -
- - -|- - O|- - -
-------------------------

-------Move 3925-------
- - -|X - -|O - -
X O O|- X -|- O -
O - O|- - -|O O -
-----------------
X O X|- - X|- O X
O X -|O - X|O X -
- - O|O - X|- - -
-----------------
- - -|- - -|X X X
X X X|X - -|- O -
- - -|- - O|- - -
-------------------------

1 1
-------Move 3934-------
- - -|X O -|O - -
X O O|- X -|- O -
O - O|- - -|O O -
-----------------
X O X|- - X|- O X
O X -|O - X|O X -
- - O|O - X|- - -
-----------------
- - -|- - -|X X X
X X X|X - -|- O -
- - -|- - O|- - -
-------------------------

-------Move 4123-------
- - -|X O -|O - -
X O O|- X -|- O -
O - O|- - X|O O -
-----------------
X O X|- - X|- O X
O X -|O - X|O X -
- - O|O - X|- - -
-----------------
- - -|- - -|X X X
X X X|X - -|- O -
- - -|- - O|- - -
-----------------------

ValueError: The game on board meta_board has ended

In [ ]:
%debug

In [ ]:
import pdb

In [ ]:
pdb.run('')

In [ ]:
hist = [(6, 2, 1),
        (2, 7, 2),
        (7, 3, 1),
        (3, 8, 2),
        (8, 3, 1),
        (3, 3, 2),
        (3, 4, 1),
        (4, 6, 2),
        (6, 4, 1),
        (4, 3, 2),
        (3, 2, 1),
        (2, 6, 2),
        (6, 6, 1),
        (5, 1, 2)]